<a href="https://colab.research.google.com/github/quaocalebfianku/Hamoye-Data-Science-Internship./blob/main/Hamoye_Stage_C_caleb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import all necessary libraries

import pandas as pd
import numpy as np                            #mathematical operation
import matplotlib.pyplot as plt               #data visualisation
import seaborn as sns                         #data visualisation

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler             #used for normalising dataset into a common scale using Min and Max.
from sklearn.model_selection import train_test_split       #used for splitting dataset into training and testing df

from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import LabelEncoder
from pandas.core.common import random_state

import lightgbm as ltb
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix

In [13]:
# Load the dataset
telco_df = pd.read_csv("/content/Hamoye Dataset/WA_Fn-UseC_-Telco-Customer-Churn.csv")
telco_df.head()
telco_df.isnull().sum()
telco_df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [14]:
# Convert 'TotalCharges' to numeric values and fill missing values with 0
telco_df['TotalCharges'] = pd.to_numeric(telco_df['TotalCharges'], errors='coerce')
telco_df['TotalCharges'].fillna(0, inplace=True)


# Convert 'Churn' column to binary values
telco_df['Churn'] = telco_df['Churn'].map({'No': 0, 'Yes': 1})

In [15]:
#Let check missing values

missing_values = pd.DataFrame({'Count':telco_df.isna().sum(),'Percentage':telco_df.isna().sum()/len(telco_df)*100})
missing_values


,Count,Percentage
customerID,0,0.0
gender,0,0.0
SeniorCitizen,0,0.0
Partner,0,0.0
Dependents,0,0.0
tenure,0,0.0
PhoneService,0,0.0
MultipleLines,0,0.0
InternetService,0,0.0
OnlineSecurity,0,0.0


In [16]:
X = telco_df.drop(columns=['Churn'])  # Features
y = telco_df['Churn']  # Target

#x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [17]:
#Split the data into an 80-20 train-test split with a random state of “1”

x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, test_size = 0.2, random_state = 1)

In [18]:
# List of categorical and numerical features
categorical_features = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService',
                        'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
                        'Contract', 'PaperlessBilling', 'PaymentMethod']

numerical_features = ['tenure', 'MonthlyCharges', 'TotalCharges']

# Scale numerical features using StandardScaler
scaler = StandardScaler()
x_train[numerical_features] = scaler.fit_transform(x_train[numerical_features])
x_test[numerical_features] = scaler.transform(x_test[numerical_features])

# One-hot encode categorical features
encoder = OneHotEncoder(sparse=False)
x_train_encoded = encoder.fit_transform(x_train[categorical_features])
x_test_encoded = encoder.transform(x_test[categorical_features])

# Convert encoded arrays back to DataFrames with column names
# One-hot encode categorical features
encoder = OneHotEncoder(sparse=False, drop='first')  # Use drop='first' to avoid multicollinearity

x_train_encoded = encoder.fit_transform(x_train[categorical_features])
x_test_encoded = encoder.transform(x_test[categorical_features])

# Get the column names for the one-hot encoded features
encoded_feature_names = encoder.get_feature_names_out(categorical_features)

# Convert encoded arrays back to DataFrames with column names
x_train_encoded_df = pd.DataFrame(x_train_encoded, columns=encoded_feature_names)
x_test_encoded_df = pd.DataFrame(x_test_encoded, columns=encoded_feature_names)

# Combine scaled numerical and one-hot encoded categorical features
x_train_final = pd.concat([x_train[numerical_features], x_train_encoded_df], axis=1)
x_test_final = pd.concat([x_test[numerical_features], x_test_encoded_df], axis=1)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [21]:
print(x_train_final.shape, x_test_final.shape)

(6753, 30) (2556, 30)


In [24]:
# Drop rows with missing values from both the features (X) and target (y) dataframes
x_train_final = x_train_final.dropna()
y_train = y_train[x_train_final.index]

# Now you can train the RandomForestClassifier with the data without missing values
clf.fit(x_train_final, y_train)

RandomForestClassifier(random_state=1)

**Question 14:**
What is the accuracy on the test set using the random forest classifier?

In [26]:
#Create a Gaussian Classifier
clf=RandomForestClassifier(random_state=1)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train_final, y_train)

y_pred=clf.predict(x_train_final)

In [ ]:
from sklearn import metrics

acc = accuracy_score(y_pred, y_test)

print("Accuracy:",round(acc,4))

**Question15:**  What is the accuracy on the test set using the xgboost classifier?



In [ ]:
#Extreme Gradient Boost classifier (XGboost)

#Define model
xgb_model = XGBClassifier(random_state = 1)

#fit model
xgb_model.fit(x_train_final, y_train)

#prediction
xgb_predicted = xgb_model.predict(x_train_final)

In [ ]:
#XGBoost accurarcy

xgb_accuracy = accuracy_score(y_test, xgb_predicted)

print("The XGboost test set accuracy is:  ", round(xgb_accuracy, 4))

**Question 16:**  What is the accuracy on the test set using the LGBM classifier?

In [29]:

#LightGBM classifier

#define model
model = ltb.LGBMClassifier(random_state=1)

#fit model
model.fit(x_train_final, y_train)
print(); print(model)

#prediction
predicted_y = model.predict(x_train_final)

Questions 17: To improve the Extra Trees Classifier, you will use the following parameters (number of estimators, minimum number of samples, minimum number of samples for leaf node and the number of features to consider when looking for the best split) for the hyperparameter grid needed to run a Randomized Cross Validation Search (RandomizedSearchCV).

n_estimators = [50, 100, 300, 500, 1000]

min_samples_split = [2, 3, 5, 7, 9]

min_samples_leaf = [1, 2, 4, 6, 8]

max_features = ['auto', 'sqrt', 'log2', None]

hyperparameter_grid = {'n_estimators': n_estimators,

                       'min_samples_leaf': min_samples_leaf,

                       'min_samples_split': min_samples_split,

                       'max_features': max_features}

Using the ExtraTreesClassifier as your estimator with cv=5, n_iter=10, scoring = 'accuracy', n_jobs = -1, verbose = 1 and random_state = 1. What are the best hyperparameters from the randomized search CV?

In [ ]:
#Define Extra Trees Classifier model
extra_tree = ExtraTreesClassifier(random_state = 1)

#Parameters to try
param = {'n_estimators': [50, 100, 300, 500, 1000],
         'min_samples_split': [2, 3, 5, 7, 9],
         'min_samples_leaf': [1, 2, 4, 6, 8],
         'max_features': ['sqrt', 'log2', 'auto', None]    #It can take four values
         }

Random_Search = RandomizedSearchCV(estimator = extra_tree, param_distributions= param , random_state = 1, n_iter=10, cv=5, scoring="accuracy",
                                   n_jobs = -1, verbose = 1)


     #Fit the model
RandomSearchResults = Random_Search.fit(x_train_final, y_train)